<If there are any questions, reach out via winston.vargo@mongodb.com>

[Find Me on LinkedIn!](https://www.linkedin.com/in/winston-vargo/)

# Welcome to Session 2
## MQL

We've already met MQL, we just didn't call it by that name. *MQL* is short for "MongoDB Query Language," and is the standard way to interface against the MongoDB database.

In a certification test, rarely will the term MQL be used. The questions are usually asked "What is the proper syntax to query using X field?" and the options are various types of MQL. The term "MQL" is an umbrella term that references a standard query syntax across the mongo shell (mongosh), drivers, the Compass GUI, etc.

Last session we finished with using MQL to insert() documents. Now, let's actually read some of the data!

## Reads

Generally, to perform a read using MQL, the syntax is as follows:

`db.findOne({filter},{projection})` <-- this returns a document

`db.find({filter},{projection})` <-- this returns multiple documents... or really a *cursor* 

A cursor is a pointer to the results of a query in memory. The client then iterates through the cursor. Its important to note that the shell automatically iterates through the first 20 documents in a cursor.

There is a shell helper function, `.toArray()` which will take the all of documents in the cursor and creat an array out of them.

So what is that `{filter}` thing? Well, that's going to take up most of this session. Generally, it is a BSON object that represents the inputs to your query. By default, the MongoDB Server returns all fields of a document. For the RDBMS people out there, it is like doing a SQL `SELECT *`. The `{projection}` object is a way to determine which fields are returned by a query. Let's take a look at some examples:

In [3]:
# First Let's connect to the DB
# Variables
mongoDB_username = ""
mongoDB_password = ""
mongoDB_connection_string = ""

In [1]:
readString = "db.movies.find({year:1991})"

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval "{readString}"

]0;mongosh mongodb://<credentials>@127.0.0.1:27017/%7BmongoDB_connection_string%7D?directConnection=true&serverSelectionTimeoutMS=2000MongoNetworkError: connect ECONNREFUSED 127.0.0.1:27017


#### Some Comments on the Above Query
- the filter object is `{year:1991}`. This is an example of the most basic query. It is a single field (`year`) equality match (1991)
- If you scroll down to the bottom of the output, note how it says "Type "it" for more." This is due to the fact that `find()` returns a *cursor*

In [5]:
readString = "db.movies.find({year:1991},{title:1,year:1})"

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval "{readString}"

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [
  {
    _id: ObjectId("573a1398f29313caabcead6a"),
    title: 'The House of Smiles',
    year: 1991
  },
  {
    _id: ObjectId("573a1398f29313caabcebadc"),
    title: 'They Call Me Macho Woman!',
    year: 1991
  },
  {
    _id: ObjectId("573a1398f29313caabcebc97"),
    title: 'Prime Suspect',
    year: 1991
  },
  {
    _id: ObjectId("573a1398f29313caabcebdf2"),
    title: 'City of the Blind',
    year: 1991
  },
  {
    _id: ObjectId("573a1398f29313caabcebf0b"),
    title: 'Armour of God 2: Operation Condor',
    year: 1991
  },
  {
    _id: ObjectId("573a1399f29313caabcec20a"),
    title: 'Perfectly Normal',
    year: 1991
  },
  {
    _id: ObjectId("573a1399f29313caabcec22b"),
    title: 'The Pit and the Pendulum',
    year: 1991
  },
  {
    _id: ObjectId("573a1399f29313caabcec3a5"),
    title: 'Thousand Pieces of Gold',
    year: 1991
  },
  {
    _id: ObjectId("573a1399f29313caabcec3d1"),
    title: 'Ultrè',
 

#### Some Comments on the Above Query
- we made a `projection` on `{title:1,year:1}` with the same `filter` as the previous example
- note that `_id` returns automatically. To disable that, you must explicitly state `{_id:0}` in the projection

## More Query Syntax
MQL is typically used in the mongo shell (which is a javascript interpreter) or in code... In both cases, it is possible to chain functions to add functionality to the query language:
- `.sort({sort})` this adds a sort to the query
- `.limit(<int> or -1)` this limits the number of documents results
- `.explain()` We will get in to this a little more later, but it returns the *query plan* for a given query

`sort` and `limit` are only supported for `find()` operations, not `findOne()`

Let's look at a few more examples:

In [6]:
readString = "db.movies.find({year:1991},{title:1,year:1,_id:0}).sort({title:1})"

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval "{readString}"

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [
  { title: '35 Up', year: 1991 },
  { year: 1991, title: 'A Brief History of Time' },
  { title: 'A Brighter Summer Day', year: 1991 },
  { title: 'A Kiss Before Dying', year: 1991 },
  { title: 'A Little Stiff', year: 1991 },
  { title: 'A Rage in Harlem', year: 1991 },
  { title: 'A Scene at the Sea', year: 1991 },
  { title: "A Woman's Tale", year: 1991 },
  { title: 'A Year Along the Abandoned Road', year: 1991 },
  { title: 'All I Want for Christmas', year: 1991 },
  { title: 'American Friends', year: 1991 },
  { title: 'And You Thought Your Parents Were Weird', year: 1991 },
  { title: 'Armour of God 2: Operation Condor', year: 1991 },
  { title: 'Assassin of the Tsar', year: 1991 },
  { title: 'At Play in the Fields of the Lord', year: 1991 },
  { title: 'Backdraft', year: 1991 },
  { year: 1991, title: 'Barton Fink' },
  { title: 'Basket Case 3', year: 1991 },
  { title: 'Beauty and the Beast', year: 1991 },


In [9]:
readString = "db.movies.find({year:1991},{title:1,year:1,_id:0}).limit(3)"

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval "{readString}"

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [
  { title: 'The House of Smiles', year: 1991 },
  { title: 'They Call Me Macho Woman!', year: 1991 },
  { title: 'Prime Suspect', year: 1991 }
]


In [7]:
readString = "db.movies.find({year:1991},{title:1,year:1,_id:0}).sort({title:1}).limit(3)"

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval "{readString}"

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [
  { title: '35 Up', year: 1991 },
  { year: 1991, title: 'A Brief History of Time' },
  { title: 'A Brighter Summer Day', year: 1991 }
]


In [19]:
readString = "db.movies.find({year:1991},{title:1,year:1,_id:0}).limit(3).sort({title:1})"

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval "{readString}"

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [
  { title: '35 Up', year: 1991 },
  { year: 1991, title: 'A Brief History of Time' },
  { title: 'A Brighter Summer Day', year: 1991 }
]


Note that the order of `sort()` and `limit()` doesn't affect the results

## `Filter` Deep Dive

- If multiple `{key:value}` pairs are passed into the `filter`, the behavior is a boolean `and`. 
- There are *[operators](https://www.mongodb.com/docs/manual/reference/operator/query/)* that enhance functionality of MQL. **It is imperative to know what most of the operators do to be a MongoDB certified developer**
- The way to do a boolean `or` is via the operator  `{$or:[]}`
 
### Operators

As I just mentioned, knowing operators are important. Here is a non comprehensive list of operators:
- `eq` - "equal": it is similar to a normal query
- `$gt/$gte/$lt/$lte` - greater [and equal] / less than [and equal]: important operator for range queries
- `{$in:[]} / {$nin:[]}` - in or not in: multiple value equality match
- `$ne` - "not equal"
- `$not`: inverses the logic of a query
- `$or`: performs a boolean or
- `$and`: performs a boolean and. usually used for [specific nesting situations](https://www.mongodb.com/docs/manual/reference/operator/query/and/#and-queries-with-multiple-expressions-specifying-the-same-operator)
- `$exists`: filters based on key existence
- `$type`: filters based on BSON type
- `$elemMatch`: intended for querying on objects within arrays

**IMPORTANT NOTE: Jupyer interprets** `$` **as a special character. so in the examples below I add a backslash before all instances of the dollar sign. We then print the query string without the backslash to help avoid confusion**

Examples:

In [62]:
readString = "db.movies.find({year:1991,cast:'Brad Pitt'},{title:1,year:1,_id:0}).limit(3)"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.find({year:1991,cast:'Brad Pitt'},{title:1,year:1,_id:0}).limit(3)

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [ { year: 1991, title: 'Johnny Suede' } ]


In [63]:
readString = "db.movies.find({year:{\$gt:1991}},{title:1,year:1,_id:0}).limit(3)"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.find({year:{$gt:1991}},{title:1,year:1,_id:0}).limit(3)

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [
  { title: 'Kate & Leopold', year: 2001 },
  { title: 'Back to the USSR - takaisin Ryssiin', year: 1992 },
  { title: 'Crime and Punishment', year: 2002 }
]


In [56]:
readString = "db.movies.find({year:{\$lt:1991}},{title:1,year:1,_id:0}).limit(3)"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.find({year:{$lt:1991}},{title:1,year:1,_id:0}).limit(3)

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [
  {
    title: 'Winsor McCay, the Famous Cartoonist of the N.Y. Herald and His Moving Comics',
    year: 1911
  },
  { title: 'In the Land of the Head Hunters', year: 1914 },
  { title: 'The Perils of Pauline', year: 1914 }
]


In [58]:
readString = "db.movies.find({year:{\$in:[1991,2001,2011]}},{title:1,year:1,_id:0}).limit(3)"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.find({year:{$in:[1991,2001,2011]}},{title:1,year:1,_id:0}).limit(3)

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [
  { title: 'Kate & Leopold', year: 2001 },
  { title: 'The House of Smiles', year: 1991 },
  { title: 'They Call Me Macho Woman!', year: 1991 }
]


In [64]:
readString = "db.movies.find({year:{\$ne:1991}},{title:1,year:1,_id:0}).limit(3)"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.find({year:{$ne:1991}},{title:1,year:1,_id:0}).limit(3)

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [
  {
    title: 'Winsor McCay, the Famous Cartoonist of the N.Y. Herald and His Moving Comics',
    year: 1911
  },
  { title: 'In the Land of the Head Hunters', year: 1914 },
  { title: 'The Perils of Pauline', year: 1914 }
]


In [67]:
readString = "db.movies.find({\$or:[{year:1920},{runtime:100}]},{title:1,year:1,runtime:1,_id:0}).limit(3)"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.find({$or:[{year:1920},{runtime:100}]},{title:1,year:1,runtime:1,_id:0}).limit(3)

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [
  { runtime: 100, title: 'Tarzan the Ape Man', year: 1932 },
  { runtime: 26, title: 'High and Dizzy', year: 1920 },
  { runtime: 100, title: 'Dekigokoro', year: 1933 }
]


In [74]:
readString = "db.movies.findOne({'runtime':{\$exists:false}},{title:1,year:1,_id:0})"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.findOne({'runtime':{$exists:false}},{title:1,year:1,_id:0})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ { title: 'Mi adorado Juan', year: 1950 }


In [75]:
readString = "db.movies.findOne({'_id':{\$type:'objectId'}},{title:1,year:1,_id:0})"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.findOne({'_id':{$type:'objectId'}},{title:1,year:1,_id:0})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ { title: 'The Great Train Robbery', year: 1903 }


## Filtering on Arrays and Sub Documents

Oftentimes, you will be filtering on keys that aren't a value, but a data structure like an array or a subdocument. The `sample_mflix.movies` collection has some examples of this:

In [82]:
# array of scalars: genres
readString = "db.movies.findOne({title:'Groundhog Day'},{title:1,year:1,_id:0, genres:1})"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.findOne({title:'Groundhog Day'},{title:1,year:1,_id:0, genres:1})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ { year: 1993, genres: [ 'Comedy', 'Romance' ], title: 'Groundhog Day' }


In [78]:
# subdocument within subdocument: tomatoes.viewer & tomatoes.critic
readString = "db.movies.findOne({title:'Groundhog Day'},{title:1,year:1,_id:0, tomatoes:1})"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.findOne({title:'Groundhog Day'},{title:1,year:1,_id:0, tomatoes:1})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ {
  year: 1993,
  title: 'Groundhog Day',
  tomatoes: {
    viewer: { rating: 3.4, numReviews: 413200, meter: 87 },
    dvd: ISODate("2002-01-29T00:00:00.000Z"),
    critic: { rating: 8, numReviews: 71, meter: 96 },
    lastUpdated: ISODate("2015-09-12T17:28:16.000Z"),
    consensus: "Smart, sweet, and inventive, Groundhog Day highlights Murray's dramatic gifts while still leaving plenty of room for laughs.",
    rotten: 3,
    production: 'Columbia Pictures',
    fresh: 68
  }
}


Another sample data collection that has some great examples of nested structures is `sample_supplies.sales`. These documents generally have a structure which is: array of objects, and each object has it's own associated array:

In [79]:
readString = "db.sales.findOne({})"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_supplies" \
--eval """{readString}"""

db.sales.findOne({})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ {
  _id: ObjectId("5bd761dcae323e45a93ccfea"),
  saleDate: ISODate("2017-06-22T09:54:14.185Z"),
  items: [
    {
      name: 'notepad',
      tags: [ 'office', 'writing', 'school' ],
      price: Decimal128("30.08"),
      quantity: 1
    },
    {
      name: 'binder',
      tags: [ 'school', 'general', 'organization' ],
      price: Decimal128("24.68"),
      quantity: 7
    },
    {
      name: 'notepad',
      tags: [ 'office', 'writing', 'school' ],
      price: Decimal128("32.1"),
      quantity: 4
    },
    {
      name: 'pens',
      tags: [ 'writing', 'office', 'school', 'stationary' ],
      price: Decimal128("31.41"),
      quantity: 3
    },
    {
      name: 'printer paper',
      tags: [ 'office', 'stationary' ],
      price: Decimal128("43.56"),
      quantity: 3
    },
    {
      name: 'pens',
      tags: [ 'writing', 'office', 'school', 'stationary' ],
      price: Decimal128("68

**concepts related to querying these shapes**
- for sub documents, use "parent.child.granchild" syntax.
- scalar queries against array fields look for *existence*
- array queries against array fields look for array equality
- `$elemMatch` will be important here. We'll look in a sec

In [81]:
# filtering on subdocument
readString = "db.movies.findOne({'tomatoes.viewer.rating':3.4},{title:1,year:1,_id:0, tomatoes:1})"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.findOne({'tomatoes.viewer.rating':3.4},{title:1,year:1,_id:0, tomatoes:1})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ {
  title: 'Winsor McCay, the Famous Cartoonist of the N.Y. Herald and His Moving Comics',
  year: 1911,
  tomatoes: {
    viewer: { rating: 3.4, numReviews: 89, meter: 47 },
    lastUpdated: ISODate("2015-08-20T18:51:24.000Z")
  }
}


In [84]:
# array existence
readString = "db.movies.findOne({genres:'Comedy'},{title:1,year:1,_id:0, genres:1})"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.findOne({genres:'Comedy'},{title:1,year:1,_id:0, genres:1})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ {
  genres: [ 'Animation', 'Short', 'Comedy' ],
  title: 'Winsor McCay, the Famous Cartoonist of the N.Y. Herald and His Moving Comics',
  year: 1911
}


In [85]:
# array equality
readString = "db.movies.findOne({genres:['Comedy']},{title:1,year:1,_id:0, genres:1})"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.findOne({genres:['Comedy']},{title:1,year:1,_id:0, genres:1})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ { genres: [ 'Comedy' ], title: 'The Strong Man', year: 1926 }


**... on `$elemMatch`**
- `$elemMatch` is a special operator for multi-key filtering among arrays of subdocuments
- consider the following query:

In [95]:
readString = "db.sales.findOne({'items.name':'binder','items.quantity':{\$gt:7}})"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_supplies" \
--eval """{readString}"""

db.sales.findOne({'items.name':'binder','items.quantity':{$gt:7}})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ {
  _id: ObjectId("5bd761dcae323e45a93ccff6"),
  saleDate: ISODate("2017-08-21T20:38:34.618Z"),
  items: [
    {
      name: 'printer paper',
      tags: [ 'office', 'stationary' ],
      price: Decimal128("14.96"),
      quantity: 10
    },
    {
      name: 'laptop',
      tags: [ 'electronics', 'school', 'office' ],
      price: Decimal128("1223.33"),
      quantity: 1
    },
    {
      name: 'notepad',
      tags: [ 'office', 'writing', 'school' ],
      price: Decimal128("18.09"),
      quantity: 3
    },
    {
      name: 'notepad',
      tags: [ 'office', 'writing', 'school' ],
      price: Decimal128("25.15"),
      quantity: 3
    },
    {
      name: 'pens',
      tags: [ 'writing', 'office', 'school', 'stationary' ],
      price: Decimal128("47.36"),
      quantity: 2
    },
    {
      name: 'notepad',
      tags: [ 'office', 'writing', 's

At first glance, it looks like I am trying to filter on a sale where in the line items is a sale of greater than 7 binders. However, the above result returned, which doesn't include a sale of greate than 7 or binders. This is becuase the way the query is interpreted, it is looking for array existence. so any array that has a value of `name:binder` and `quantity:$gt:7` is returned.

This is where `$elemMatch` comes in. look at this query:

In [96]:
readString = "db.sales.findOne({items:{\$elemMatch:{name:'binder',quantity:{\$gt:7}}}})"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_supplies" \
--eval """{readString}"""

db.sales.findOne({items:{$elemMatch:{name:'binder',quantity:{$gt:7}}}})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ {
  _id: ObjectId("5bd761dcae323e45a93ccff9"),
  saleDate: ISODate("2017-11-12T20:30:15.045Z"),
  items: [
    {
      name: 'notepad',
      tags: [ 'office', 'writing', 'school' ],
      price: Decimal128("9.91"),
      quantity: 3
    },
    {
      name: 'pens',
      tags: [ 'writing', 'office', 'school', 'stationary' ],
      price: Decimal128("22.01"),
      quantity: 4
    },
    {
      name: 'backpack',
      tags: [ 'school', 'travel', 'kids' ],
      price: Decimal128("57.14"),
      quantity: 2
    },
    {
      name: 'binder',
      tags: [ 'school', 'general', 'organization' ],
      price: Decimal128("21.92"),
      quantity: 7
    },
    {
      name: 'binder',
      tags: [ 'school', 'general', 'organization' ],
      price: Decimal128("20.12"),
      quantity: 9
    },
    {
      name: 'laptop',
      tags: [ 'electronics', 's

The above query only returns due to the line item that has `name:binder` and `quantity:9`

## `countDocuments()`

One other query to be aware of is `db.collection.countDocuments({filter})`

it accepts all of the filters we've shown above, however the result is simply an integer which represents how many documents would be returned by the query

**This is the new version of MDB legacy `db.collection.count({})`. `count()` is deprecated. This will probably come up on the exam**

In [97]:
readString = "db.movies.countDocuments({'runtime':{\$exists:false}})"

print(readString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use sample_mflix" \
--eval """{readString}"""

db.movies.countDocuments({'runtime':{$exists:false}})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ 439


## Deletes

Deletes are basically like `find`s in that they accept a filter a parameter. The difference is instead of returning documents, delete will.... delete them (believe it or not)

2 main delete interfaces:
- `db.collection.deleteOne({filter},{options})`
- `db.collection.deleteMany({filter},{options})`

Options include things like write concern (a mongoDB parameter for durability / performance tradeoffs) collation (for language support) and hint (index hinting)

I won't spend too long here because we just spent so much time with filters and operators

In [109]:
# inserting docs to Delete
insertString = "db.testCollection2.insertMany([{a:1},{a:2},{a:3},{a:4},{a:5}])"
deleteString = "db.testCollection2.deleteMany({})"

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{deleteString}" \
--eval "{insertString}"

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ {
  acknowledged: true,
  insertedIds: {
    '0': ObjectId("64d3d14a4652eacdd2a8076e"),
    '1': ObjectId("64d3d14a4652eacdd2a8076f"),
    '2': ObjectId("64d3d14a4652eacdd2a80770"),
    '3': ObjectId("64d3d14a4652eacdd2a80771"),
    '4': ObjectId("64d3d14a4652eacdd2a80772")
  }
}


In [110]:
# deleteOne
deleteString = "db.testCollection2.deleteOne({a:1})"

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{deleteString}"

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ { acknowledged: true, deletedCount: 1 }


In [111]:
# deleteOne
deleteString = "db.testCollection2.deleteOne({a:6})"

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{deleteString}"

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ { acknowledged: true, deletedCount: 0 }


In [112]:
findString = "db.testCollection2.find({})"

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{findString}"

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [
  { _id: ObjectId("64d3d14a4652eacdd2a8076f"), a: 2 },
  { _id: ObjectId("64d3d14a4652eacdd2a80770"), a: 3 },
  { _id: ObjectId("64d3d14a4652eacdd2a80771"), a: 4 },
  { _id: ObjectId("64d3d14a4652eacdd2a80772"), a: 5 }
]


In [113]:
# deleteOne
deleteString = "db.testCollection2.deleteOne({a:{\$lte:5}})"

print(deleteString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{deleteString}"

db.testCollection2.deleteOne({a:{$lte:5}})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ { acknowledged: true, deletedCount: 1 }


In [ ]:
#comment on natural order

In [114]:
findString = "db.testCollection2.find({})"

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{findString}"

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [
  { _id: ObjectId("64d3d14a4652eacdd2a80770"), a: 3 },
  { _id: ObjectId("64d3d14a4652eacdd2a80771"), a: 4 },
  { _id: ObjectId("64d3d14a4652eacdd2a80772"), a: 5 }
]


In [115]:
# deleteMany
deleteString = "db.testCollection2.deleteMany({a:{\$gte:4}})"

print(deleteString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{deleteString}"

db.testCollection2.deleteMany({a:{$gte:4}})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ { acknowledged: true, deletedCount: 2 }


In [116]:
findString = "db.testCollection2.find({})"

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{findString}"

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ [ { _id: ObjectId("64d3d14a4652eacdd2a80770"), a: 3 } ]


## Updates

Buckle up. Updates are the most nuanced of the queries that will show up on the developer certification exam.

generally there are 3 main update commands:
- `db.collection.updateOne({filter},{update},{options})`
- `db.collection.updateMany({filter},{update},{options})`
- `db.collection.findAndModify({query document})`

I will focus on the first 2 for now  because they have similar syntax.

- `{filter}` - same thing we've been talking about all class
- `{update}` - either a replacement or a modification. We'll spend some time here
- `{options}` - notably, `{upsert:true|false}` which we will also discuss

### The `{update}` Object
- if this is a document, it will act as a REPLACE OPERATION <-- this is actually legacy behavior and oftentimes will error out
- similar to the filter operators, there are update operators that specify the type of modification desired.

**Common Update Operators**
- `$set` - adds / updates fields
- `$unset` - removes fields
- `$inc` - increments a field
- `$pull / $push` - removes or inserts into an array

Let's look at some examples:

In [119]:
# inserting docs to Update
deleteString = "db.testCollection3.deleteMany({})"
insertString = "db.testCollection3.insertMany([{a:1},{a:2},{a:3},{a:4},{a:5}])"

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{deleteString}" \
--eval "{insertString}"

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ {
  acknowledged: true,
  insertedIds: {
    '0': ObjectId("64d3dc5c148ecd164655546c"),
    '1': ObjectId("64d3dc5c148ecd164655546d"),
    '2': ObjectId("64d3dc5c148ecd164655546e"),
    '3': ObjectId("64d3dc5c148ecd164655546f"),
    '4': ObjectId("64d3dc5c148ecd1646555470")
  }
}


In [12]:
updateString = "db.testCollection3.updateOne({a:1},{\$set:{b:3}})"
readString = "db.testCollection3.findOne({a:1})"

print(updateString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{updateString}" \
--eval "{readString}"

db.testCollection3.updateOne({a:1},{$set:{b:3}})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ { _id: ObjectId("64d3dc5c148ecd164655546c"), a: 1, b: 3 }


In [13]:
updateString = "db.testCollection3.updateOne({a:1,b:3},{\$unset:{b:3}})"
readString = "db.testCollection3.findOne({a:1})"

print(updateString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{updateString}" \
--eval "{readString}"

db.testCollection3.updateOne({a:1,b:3},{$unset:{b:3}})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ { _id: ObjectId("64d3dc5c148ecd164655546c"), a: 1 }


In [15]:
# Re run this multiple times and see what happens!

updateString = "db.testCollection3.updateOne({a:1},{\$inc:{b:3}})"
readString = "db.testCollection3.findOne({a:1})"

print(updateString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{updateString}" \
--eval "{readString}"

db.testCollection3.updateOne({a:1},{$inc:{b:3}})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ { _id: ObjectId("64d3dc5c148ecd164655546c"), a: 1, b: 6 }


In [18]:
# Re run this multiple times and see what happens!

updateString = "db.testCollection3.updateOne({a:1},{\$push:{c:3}})"
readString = "db.testCollection3.findOne({a:1})"

print(updateString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{updateString}" \
--eval "{readString}"

db.testCollection3.updateOne({a:1},{$push:{c:3}})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ {
  _id: ObjectId("64d3dc5c148ecd164655546c"),
  a: 1,
  b: 6,
  c: [ 3, 3, 3 ]
}


In [22]:
# Re run this multiple times and see what happens!

updateString = "db.testCollection3.updateOne({a:1},{\$push:{c:3,d:3},\$inc:{b:3}})"
readString = "db.testCollection3.findOne({a:1})"

print(updateString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{updateString}" \
--eval "{readString}"

db.testCollection3.updateOne({a:1},{$push:{c:3,d:3},$inc:{b:3}})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ {
  _id: ObjectId("64d3dc5c148ecd164655546c"),
  a: 1,
  b: 18,
  c: [
    3, 3, 3, 3,
    3, 3, 3
  ],
  d: [ 3 ]
}


### Thoughts and Reflections

- you can put as many operators in a single update statement as you want. These are ATOMIC
- you can put as many fields as you want to update in an operator

### Upserting

The normal behavior to experience on update is that if no documents are matched by the filter, nothing happens. However, if you set `{upsert:true}` you can insert if no documents match the filter

In [23]:
updateString = "db.testCollection3.updateOne({a:6},{\$inc:{b:2}})"
readString = "db.testCollection3.findOne({a:6})"

print(updateString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{updateString}" \
--eval "{readString}"

db.testCollection3.updateOne({a:6},{$set:{b:2}})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ null


In [26]:
# Re run this multiple times and see what happens!

updateString = "db.testCollection3.updateOne({a:6},{\$inc:{b:2}},{upsert:true})"
readString = "db.testCollection3.findOne({a:6})"

print(updateString.replace("\\", ""))
print("")

!mongosh {mongoDB_connection_string} --quiet  --username {mongoDB_username} -p {mongoDB_password} \
--eval "use testDb" \
--eval "{updateString}" \
--eval "{readString}"

db.testCollection3.updateOne({a:6},{$inc:{b:2}},{upsert:true})

]0;mongosh mongodb+srv://<credentials>@devcert.6ngvd.mongodb.net/ { _id: ObjectId("64dbe41b15506878d86f3fb4"), a: 6, b: 6 }
